<a href="https://colab.research.google.com/github/amaslov455/nlp_project/blob/main/sst_tocsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp36-none-any.whl size=37070 sha256=c906c967661d52ae56c4512ff8cf6210abc591ecd1a98e2aecd964a9ab8ae827
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [ ]:
import pytreebank
import pandas as pd

In [ ]:
dataset = pytreebank.load_sst()

In [ ]:
dataset.keys()

dict_keys(['train', 'test', 'dev'])

In [ ]:
dataset['train'][0]

In [ ]:
def create_df_from_treebank(input_dataset):
  dict_ = {}
  dict_['sentence'] = []
  dict_['santiment'] = []

  list_santiments = ["very_negative", "negative", "neutral", "positive", "very_positive"]

  for part in input_dataset:
    label, sentence = part.to_labeled_lines()[0]

    dict_['sentence'].append(sentence)
    dict_['santiment'].append(list_santiments[label])

  df = pd.DataFrame.from_dict(dict_)
  return df

In [ ]:
df_train = create_df_from_treebank(dataset['train'])
df_test = create_df_from_treebank(dataset['test'])

In [ ]:
df_train

,sentence,santiment
0,The Rock is destined to be the 21st Century 's...,positive
1,The gorgeously elaborate continuation of `` Th...,very_positive
2,Singer/composer Bryan Adams contributes a slew...,positive
3,You 'd think by now America would have had eno...,neutral
4,Yet the act is still charming here .,positive
...,...,...
8539,A real snooze .,very_negative
8540,No surprises .,negative
8541,We 've seen the hippie-turned-yuppie plot befo...,positive
8542,Her fans walked out muttering words like `` ho...,very_negative


In [ ]:
df_test

,sentence,santiment
0,Effective but too-tepid biopic,neutral
1,If you sometimes like to go to the movies to h...,positive
2,"Emerges as something rare , an issue movie tha...",very_positive
3,The film provides some great insight into the ...,neutral
4,Offers that rare combination of entertainment ...,very_positive
...,...,...
2205,An imaginative comedy/thriller .,positive
2206,"( A ) rare , beautiful film .",very_positive
2207,( An ) hilarious romantic comedy .,very_positive
2208,Never ( sinks ) into exploitation .,positive


In [ ]:
df_train.to_csv('/content/drive/MyDrive/diplom_project/train.csv', index = False)
df_test.to_csv('/content/drive/MyDrive/diplom_project/test.csv', index = False)